In [ ]:
#@markdown ## Google Drive integration
#@markdown **Check to automatically save the created image to Google Drive.**
Google_Drive = False #@param {type:"boolean"}

#@markdown **Check to save the initial model (checkpoint) to Google Drive.**
#@markdown <div><font color="red">The model file size (2GB ~ 7GB) is large, so please check the Google Drive capacity</div>
#@markdown <div><font color="red">Instead, once you receive it, the next execution will be faster</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown **WebUI Version**
TagVersion = 'v1.6.0' #@param ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0"]

#@markdown -----
#@markdown *tunneling*

#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

#@markdown **Localtunnel**
Localtunnel = False #@param {type:"boolean"}

#@markdown -----
#@markdown *No need to set anything below*

#@markdown **Connect folder to Google Drive**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **Deskripsi Folder** : You can create it in advance, or if it does not exist, it will be created automatically.
#@markdown * webui/output : Created images are saved.
#@markdown * webui/checkpoint : You can read it by entering a model (checkpoint).
#@markdown * webui/lora : You can read it by inserting LoRA.
#@markdown * webui/lycoris : You can read it by entering LyCORIS.
#@markdown * webui/embedding : You can read it by adding embedding (Textual Inversion).
#@markdown * webui/hyperwork : You can read it by inserting Hyperworks.
#@markdown * webui/wildcards : You can read it by inserting wildcards.

#@markdown **Initial model (checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/108576?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
#@markdown **Initial model file name**
Checkpoint_Filename = 'absolutereality_v16.safetensors' #@param {type:"string"}

#@markdown -----
#@markdown *Additional extension*

#@markdown **Uncheck if you do not use ControlNet**
ControlNet = True #@param {type:"boolean"}

#@markdown **Uncheck After Detailer(adetailer) if you do not use it**
ADetailer = True #@param {type:"boolean"}

#@markdown **Uncheck if you do not use Decect Detailer (ddetailer)**
DDetailer = False #@param {type:"boolean"}

#@markdown **Uncheck if you do not use Segment Anything**
SegmentAnything = False #@param {type:"boolean"}

#@markdown **Uncheck mov2mov if you do not use it (only works on webui version 1.3 or lower)**
Mov2mov = False #@param {type:"boolean"}

Workspace = 'ui'
NotebookVersion = 'nightly'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion/google'

!apt -y install -qq aria2

%cd /content
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/install_{NotebookVersion}.sh -O /content/install.sh
!bash /content/install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything} {Mov2mov}

# Google Drive connection
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/link_google_drive.sh -O /content/link_google_drive.sh
!bash /content/link_google_drive.sh {Workspace} {Google_Drive} {Google_Drive_Dir} {NotebookVersion}

# customizing
%cd /content/{Workspace}

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' /content/{Workspace}/config.json

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/ESRGAN -o 4x-UltraSharp.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/{Workspace}/models/VAE -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/63006?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o LowRA.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16" -d /content/{Workspace}/models/Lora -o blindbox_V1Mix.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d /content/{Workspace}/embeddings -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/embeddings/-o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/embeddings/-o bad_prompt_version2.pt
if Localtunnel:
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

if Ngrok_Key:
  !python launch.py --xformers --no-half-vae --theme dark --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue
